Let's talk about GridSearchCV.

In [1]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from PIL import Image
from io import BytesIO as sio
import IPython.display as ipd
import seaborn as sns
from sklearn.model_selection import GridSearchCV

In [2]:
# load the data
(x_tr, y_tr), (x_te, y_te) = mnist.load_data()

# the inputs are given in 2d arrays, and we need them to be 1d arrays
x_tr_2d = [x.reshape(-1,) for x in x_tr]
x_te_2d = [x.reshape(-1,) for x in x_te]

In [3]:
# define the grid search parameters
activation = ['logistic', 'tanh', 'relu']
learning_rate = ['constant', 'adaptive']
learning_rate_init = np.linspace(0.0005,0.002,4)
param_grid = dict(activation = activation,
                  learning_rate = learning_rate,
                  learning_rate_init = learning_rate_init)

In [9]:
# instantiate the model and define the grid to search
nn = MLPClassifier()
grid = GridSearchCV(estimator=nn, param_grid=param_grid, n_jobs=-1, cv=3)

In [10]:
# search the grid (train models)
grid_result = grid.fit(x_tr_2d, y_tr)

In [12]:
grid_result.__dict__

{'scoring': None,
 'estimator': MLPClassifier(),
 'n_jobs': -1,
 'refit': True,
 'cv': 3,
 'verbose': 0,
 'pre_dispatch': '2*n_jobs',
 'error_score': nan,
 'return_train_score': False,
 'param_grid': {'activation': ['logistic', 'tanh', 'relu'],
  'learning_rate': ['constant', 'adaptive'],
  'learning_rate_init': array([0.0005, 0.001 , 0.0015, 0.002 ])},
 'multimetric_': False,
 'best_index_': 0,
 'best_score_': 0.9600999999999998,
 'best_params_': {'activation': 'logistic',
  'learning_rate': 'constant',
  'learning_rate_init': 0.0005},
 'best_estimator_': MLPClassifier(activation='logistic', learning_rate_init=0.0005),
 'refit_time_': 113.855144739151,
 'scorer_': <function sklearn.metrics._scorer._passthrough_scorer(estimator, *args, **kwargs)>,
 'cv_results_': {'mean_fit_time': array([568.52149256, 274.35483925, 291.97218657, 193.56794063,
         481.94801148, 411.88734667, 381.73154418, 300.25192356,
         404.36706217, 313.13908394, 264.94753361, 303.04226224,
         623.55

In [11]:
print(f'best score: {grid_result.best_score_:f}, using')
for key in grid_result.best_params_.keys():
    print(f'   {key}: {grid_result.best_params_[key]}')
print('\n')

best score: 0.960100, using
   activation: logistic
   learning_rate: constant
   learning_rate_init: 0.0005




In [13]:
grid_result.best_params_

{'activation': 'logistic',
 'learning_rate': 'constant',
 'learning_rate_init': 0.0005}

In [14]:
grid_result.cv_results_

{'mean_fit_time': array([568.52149256, 274.35483925, 291.97218657, 193.56794063,
        481.94801148, 411.88734667, 381.73154418, 300.25192356,
        404.36706217, 313.13908394, 264.94753361, 303.04226224,
        623.55226556, 372.53001213, 400.42738565, 384.04465294,
        273.14909561, 304.95270626, 226.60083524, 252.68672784,
        228.24274429, 242.83898481, 191.68539405, 131.65336839]),
 'std_fit_time': array([ 89.04210839,  19.84223886, 105.33059236,  53.42036189,
        131.08088928, 155.04616701,  35.54175058,  90.02494035,
         40.930178  ,  50.58573851,  29.53912366, 117.46234712,
         24.17668383,  70.07003841,  27.29924502, 182.1455871 ,
         75.3006973 ,  66.02705677,  67.96936352,  30.17315173,
         38.31057605,   6.70191015,  10.44965956,  30.17278613]),
 'mean_score_time': array([0.67359789, 0.73863864, 0.59037169, 0.55236673, 0.62993272,
        0.61560551, 0.55342658, 0.53609331, 0.58941746, 0.58536275,
        0.52012976, 0.5903155 , 0.540223

Let's see all model configurations and results.

In [15]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
res = []
for mean, stdev, param in zip(means, stds, params):
    res.append((round(mean,4), round(stdev,5), *param.values()))
res.sort(reverse=True)
res

[(0.9601, 0.00084, 'logistic', 'constant', 0.0005),
 (0.96, 0.0031, 'relu', 'constant', 0.0005),
 (0.9595, 0.00195, 'relu', 'adaptive', 0.001),
 (0.9594, 0.0013, 'relu', 'adaptive', 0.0005),
 (0.959, 0.003, 'relu', 'constant', 0.001),
 (0.9585, 0.00204, 'logistic', 'adaptive', 0.0005),
 (0.9545, 0.00096, 'relu', 'adaptive', 0.0015),
 (0.9534, 0.00134, 'tanh', 'adaptive', 0.0005),
 (0.9531, 0.00343, 'relu', 'constant', 0.0015),
 (0.951, 0.00383, 'logistic', 'adaptive', 0.001),
 (0.9507, 0.00332, 'tanh', 'constant', 0.0005),
 (0.9465, 0.00119, 'logistic', 'constant', 0.001),
 (0.9454, 0.00275, 'relu', 'adaptive', 0.002),
 (0.9443, 0.00198, 'relu', 'constant', 0.002),
 (0.9424, 0.0038, 'logistic', 'adaptive', 0.0015),
 (0.9409, 0.00573, 'logistic', 'constant', 0.0015),
 (0.9367, 0.00318, 'tanh', 'constant', 0.001),
 (0.9355, 0.0038, 'tanh', 'adaptive', 0.001),
 (0.9334, 0.00535, 'logistic', 'adaptive', 0.002),
 (0.9313, 0.00638, 'logistic', 'constant', 0.002),
 (0.9307, 0.00072, 'tanh', '

### Convolutional Neural Networks (CNNs)

Let's explore CNN components. First we see how convolution layers work.

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D

In [ ]:
# create an array to which we'll apply the filter
data = np.asarray([0, 0, 0, 1, 1, 1, 1, 0, 0, 0])

keras expects 1D arrays to come in a 3D package. The first dimension is for the sample (i.e. one sample among many), the second dimension is for the length of each array, and the third dimension is for the channel (as in color).

In [ ]:
data = data.reshape(1, 10, 1)

Sequential is a class from keras that lets us stack together layer one-at-a-time into a single model. Conv1D and Conv2D are convolusional layers for application to sets of 1D and 2D arrays respectively.

In [ ]:
model = Sequential()

We'll use one filter with a length of 3, so we'll add one convolution layer to our model.

In [ ]:
model.add(Conv1D(1, 3, input_shape=(10, 1)))

In [ ]:
# define a vertical line detector. The final weight is a bias., added to all outputs.
weights = [np.asarray([[[0]],[[1]],[[0]]]), np.asarray([0])]

In [ ]:
# store the weights in the model
model.set_weights(weights)

In [ ]:
print(model.get_weights())

In [ ]:
# apply filter to input data
yhat = model.predict(data)
print(yhat)

Of course, we're really just detecting 1s, and not detecting edges.

In [ ]:
weights = [np.asarray([[[-1]],[[1]],[[0]]]), np.asarray([0])]
model.set_weights(weights)
yhat = model.predict(data)
print(yhat)

There are some edges.

Now we'll apply a filter to a two-dimensional array.

In [ ]:
# define input data
data = np.asarray(
    [[[0, 0, 0, 1, 1, 0, 0, 0],
      [0, 0, 0, 1, 1, 0, 0, 0],
      [0, 0, 0, 1, 1, 0, 0, 0],
      [0, 0, 0, 1, 1, 0, 0, 0],
      [0, 0, 0, 1, 1, 0, 0, 0],
      [0, 0, 0, 1, 1, 0, 0, 0],
      [0, 0, 0, 1, 1, 0, 0, 0],
      [0, 0, 0, 1, 1, 0, 0, 0]],
     
     [[0, 0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0, 0],
      [1, 1, 1, 1, 1, 1, 1, 1],
      [1, 1, 1, 1, 1, 1, 1, 1],
      [1, 1, 1, 1, 1, 1, 1, 1],
      [0, 0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0, 0]]])

data = data.reshape(2, 8, 8, 1)

In [ ]:
# create model
model = Sequential()
model.add(Conv2D(1, (3,3), input_shape=(8, 8, 1)))

In [ ]:
# define a vertical line detector
detector = [[[[-1]],[[1]],[[0]]],
            [[[-1]],[[1]],[[0]]],
            [[[-1]],[[1]],[[0]]]]
weights = [np.asarray(detector), np.asarray([0.0])]
# store the weights in the model
model.set_weights(weights)

In [ ]:
# apply filter to input data
yhat = model.predict(data)

We have a 2x6x6x1 output.

In [ ]:
yhat.shape

We can arrange the output into a 2D grids for nice printing.

In [ ]:
# first input grid, vertical edge detector
print(yhat[0,:,:,0])

In [ ]:
# second input grid (horizontal stripe), vertical edge detector
print(yhat[1,:,:,0])

See that our filter identifies vertical edges but ignores horizontal edges. We can make a filter to detect horizontal edges and ignore vertical edges.

In [ ]:
# let's include two filters with this model
model = Sequential()
model.add(Conv2D(2, (3,3), input_shape=(8, 8, 1)))

# define vertical and horizontal line detectors
detector = [[[[-1,-1]],[[1,-1]],[[0,-1]]],
            [[[-1,1]],[[1,1]],[[0,1]]],
            [[[-1,0]],[[1,0]],[[0,0]]]]
            
# our bias vector must have two components
weights = [np.asarray(detector), np.asarray([0.0,0])]
model.set_weights(weights)

In [ ]:
# apply filter to input data
yhat = model.predict(data)

In [ ]:
yhat.shape

In [ ]:
# see the effects of the filters on the first input array
yhat[:,:,:,0]

In [ ]:
# see the effects of the filters on the second input array
yhat[:,:,:,1]

Next we discuss [pooling](https://machinelearningmastery.com/pooling-layers-for-convolutional-neural-networks/).

Pooling is an operation done to a numeric grid (e.g. image), similar to applying a convolution. However, the pooling filter is usually 2x2 and applied with a stride of 2 (no overlap between applications). For each 2x2 subgrid of our image, a scalar is produced. Max pooling takes the max of of the four pixels, while average pooling takes the average of the four pixels. The image is thus down sampled by a factor of 4.

The utility of pooling is that sometimes features we're trying to detect may change their location slightly within an image. The down sampling involved in pooling gives us a margin of error for the location of features we're detecting.

In [ ]:
# example of max pooling
from numpy import asarray
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
# define input data
data = [[0, 0, 0, 1, 1, 1, 0, 0],
 [0, 0, 0, 1, 1, 1, 0, 0],
 [0, 0, 0, 1, 1, 1, 0, 0],
 [0, 0, 0, 1, 1, 1, 0, 0],
 [0, 0, 0, 1, 1, 1, 0, 0],
 [0, 0, 0, 1, 1, 1, 0, 0],
 [0, 0, 0, 1, 1, 1, 0, 0],
 [0, 0, 0, 1, 1, 1, 0, 0]]
data = asarray(data)
data = data.reshape(1, 8, 8, 1)

# create model
model = Sequential()
model.add(Conv2D(1, (3,3), activation='relu', input_shape=(8, 8, 1)))

# add a pooling layer
model.add(MaxPooling2D())
# model.add(AveragePooling2D())

# summarize model
model.summary()
# define a vertical line detector
detector = [[[[0]],[[1]],[[0]]],
            [[[0]],[[1]],[[0]]],
            [[[0]],[[1]],[[0]]]]
weights = [asarray(detector), asarray([0.0])]
# store the weights in the model
model.set_weights(weights)
# apply filter to input data
yhat = model.predict(data)
# enumerate rows
for r in range(yhat.shape[1]):
    # print each column in the row
    print([yhat[0,r,c,0] for c in range(yhat.shape[2])])